In [1]:
import talib
import numpy as np
import pandas as pd
import random
from collections import deque
import pandas_datareader as pdr
import datetime

In [2]:
start = datetime.datetime(2014, 1, 2)

vix = pdr.DataReader("VIXCLS", "fred", start)
vix['Date'] = vix.index
vix.reset_index(drop=True, inplace=True)
vix.rename(columns={'VIXCLS': 'vix', 'Date': 'Day'}, inplace=True)
vix.Day = pd.to_datetime(vix.Day).dt.date

print(vix.head())
print(vix.tail())

     vix         Day
0  14.23  2014-01-02
1  13.76  2014-01-03
2  13.55  2014-01-06
3  12.92  2014-01-07
4  12.87  2014-01-08
        vix         Day
1978  18.04  2021-08-03
1979  17.97  2021-08-04
1980  17.28  2021-08-05
1981  16.15  2021-08-06
1982  16.72  2021-08-09


In [3]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [4]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_D1_2014-2021.csv')
df.Date = pd.to_datetime(df.Date)
df['Day'] = df.Date.dt.date
df = pd.merge(df, vix, how='inner', on='Day')
df.drop(['Day'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

df_small = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2017-2021.csv')
df_small.Date = pd.to_datetime(df_small.Date)
df_small.reset_index(drop=True, inplace=True)

df_rth = pd.read_csv(file_path + 'Oanda_Gbp_Usd_D1_RTH_2014-2021.csv')
df_rth.Date = pd.to_datetime(df_rth.Date)

In [5]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2014-07-29 21:00:00   1.69425   1.69546  1.68884    1.69100   1.69447   
1 2014-07-30 21:00:00   1.69110   1.69259  1.68562    1.68826   1.69144   
2 2014-07-31 21:00:00   1.68820   1.68921  1.68029    1.68167   1.68875   
3 2014-08-04 21:00:00   1.68582   1.68876  1.68425    1.68846   1.68637   
4 2014-08-05 21:00:00   1.68856   1.68856  1.68143    1.68505   1.68873   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close    vix  
0   1.69563  1.68902    1.69154   1.69436   1.69554  1.68896    1.69127  13.28  
1   1.69275  1.68581    1.68874   1.69127   1.69267  1.68572    1.68850  13.33  
2   1.68950  1.68176    1.68257   1.68848   1.68930  1.68129    1.68212  16.95  
3   1.68964  1.68457    1.68883   1.68610   1.68888  1.68449    1.68864  15.12  
4   1.68886  1.68226    1.68558   1.68864   1.68864  1.68218    1.68532  16.87  
---------------------------------------------------------------

In [6]:
print(df_small.head())
print('-' * 75)
print(df_small.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2017-07-30 21:00:00   1.31167   1.31263  1.31167    1.31223   1.31275   
1 2017-07-30 21:05:00   1.31216   1.31261  1.31215    1.31217   1.31324   
2 2017-07-30 21:10:00   1.31262   1.31262  1.31215    1.31217   1.31370   
3 2017-07-30 21:15:00   1.31210   1.31253  1.31210    1.31253   1.31318   
4 2017-07-30 21:20:00   1.31263   1.31263  1.31263    1.31263   1.31371   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.31371  1.31275    1.31331   1.31221   1.31317  1.31221    1.31277  
1   1.31369  1.31323    1.31325   1.31270   1.31315  1.31269    1.31271  
2   1.31370  1.31323    1.31325   1.31316   1.31316  1.31269    1.31271  
3   1.31361  1.31318    1.31361   1.31264   1.31307  1.31264    1.31307  
4   1.31371  1.31371    1.31371   1.31317   1.31317  1.31317    1.31317  
---------------------------------------------------------------------------
                      Date  B

In [7]:
def get_on_close_up(row):
    valid_rows = df_rth[df_rth.Date >= row.Date]
    if len(valid_rows) == 0:
        return np.NaN

    rth_row = valid_rows.iloc[0, :]
    rth_open = rth_row['Mid_Open']
    on_open = row['Mid_Open']

    return 1 if rth_open > on_open else 0

def get_range_as_percentage(row):
    mid_open, mid_high, mid_low = row[['Mid_Open', 'Mid_High', 'Mid_Low']]
    day_range = mid_high - mid_low

    return day_range / mid_open

In [8]:
# df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
# df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
# df['ema50'] = talib.EMA(df['Mid_Close'], timeperiod=50)
# df['ema30'] = talib.EMA(df['Mid_Close'], timeperiod=30)
# df['ema25'] = talib.EMA(df['Mid_Close'], timeperiod=25)
# df['ema20'] = talib.EMA(df['Mid_Close'], timeperiod=20)
# df['ema15'] = talib.EMA(df['Mid_Close'], timeperiod=15)
# df['slowk'], df['slowd'] = talib.STOCH(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
# df['atr'] = talib.ATR(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=500)
# # df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
# df.dropna(inplace=True)
# df.reset_index(drop=True, inplace=True)
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['ema50'] = talib.EMA(df['Mid_Close'], timeperiod=50)
df['ema30'] = talib.EMA(df['Mid_Close'], timeperiod=30)
df['ema25'] = talib.EMA(df['Mid_Close'], timeperiod=25)
df['ema20'] = talib.EMA(df['Mid_Close'], timeperiod=20)
df['ema15'] = talib.EMA(df['Mid_Close'], timeperiod=15)
df['atr'] = talib.ATR(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=500)
df['on_close_up'] = df.apply(get_on_close_up, axis=1)
df['range_as_percentage'] = df.apply(get_range_as_percentage, axis=1)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)


In [9]:
print(df.head())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2016-12-27 22:00:00   1.22667   1.22959  1.21999    1.22235   1.22767   
1 2016-12-28 22:00:00   1.22289   1.22736  1.22074    1.22570   1.22389   
2 2016-12-29 22:00:00   1.22552   1.23871  1.22437    1.23368   1.22652   
3 2017-01-03 22:00:00   1.22312   1.23518  1.22201    1.23193   1.22412   
4 2017-01-04 22:00:00   1.23190   1.24314  1.22694    1.24137   1.23290   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  ...    vix    ema200     ema50  \
0   1.22984  1.22020    1.22300   1.22717  ...  11.99  1.341551  1.254250   
1   1.22757  1.22109    1.22670   1.22339  ...  12.95  1.340403  1.253150   
2   1.23896  1.22477    1.23468   1.22602  ...  13.37  1.339346  1.252406   
3   1.23549  1.22231    1.23239   1.22362  ...  12.85  1.337141  1.250535   
4   1.24336  1.22713    1.24237   1.23240  ...  11.85  1.336193  1.250195   

      ema30     ema25     ema20     ema15       atr  on_close_up  \
0  1.245386  1.243

In [10]:
value_per_pip = 1.0
max_pips_to_risk = 0.0100
amounts_per_day = [-4, -5, -6]

In [17]:
df = df[df.Date >= '2020-07-30 21:00:00']
df.reset_index(drop=True, inplace=True)

In [23]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_simulation(risk_reward_ratio, pullback_cushion, atr_percentage, last_x_bars, fractal_distance, bar_length, ema_val, max_open_trades=1):
    pullback_cushion /= 10000
    fractal_distance /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000 / max_open_trades
    trades = deque(maxlen=max_open_trades)

    def iterate_thru_small_data(trade):
        def get_end_date(j):
          return df_small.loc[df_small.index[j + 1], 'Date'] if j + 1 < len(df_small) else None

        j = df_small[df_small.Date >= trade['start_date']].index[0]

        nonlocal reward
        nonlocal day_fees
        nonlocal n_losses
        nonlocal n_wins
        nonlocal curr_win_streak
        nonlocal curr_loss_streak
        nonlocal win_streak
        nonlocal loss_streak

        while j < len(df_small):
          curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_date = \
          df_small.loc[df_small.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low',
                                         'Ask_Close', 'Date']]

          if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
              trade['stop_loss'] = curr_bid_high - trade['pips_risked']

          if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
            trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

          if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          j += 1


    i = 2

    while i < len(df):
        if i < last_x_bars:
            i += 1
            continue

        curr_date = df.loc[df.index[i], 'Date']

        # Remove any expired trades -----------------------------------------
        trades_to_remove = []

        for trade in trades:
          if trade['end_date'] is not None and trade['end_date'] >= curr_date:
            trades_to_remove.append(trade)

        for trade in trades_to_remove:
          trades.remove(trade)
        # -------------------------------------------------------------------

        ema200 = df.loc[df.index[i - 1], 'ema200']
        pullback = df.loc[df.index[i - 1], ema_val]
        atr = df.loc[df.index[i - 1], 'atr']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = (spread / atr) <= atr_percentage
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        previous_ask_lows = []
        previous_bid_highs = []
        curr_ml = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh - curr_ml)
        vix, range_percentage, close_up = df.loc[df.index[i - 1], ['vix', 'range_as_percentage', 'on_close_up']]
        curr_pred_buy = vix >= 18 and range_percentage >= 0.01 and close_up == 1

        for ind in range(i - last_x_bars, i):
          previous_ask_lows.append(df.loc[df.index[ind], 'Ask_Low'])
          previous_bid_highs.append(df.loc[df.index[ind], 'Bid_High'])

        process_trade = False

        # if beep_boop2 == 1 and beep_boop1 == 1 and curr_ml > ema200 and enough_volatility and len(trades) < max_open_trades and curr_al <= min(previous_ask_lows) and curr_bar_length <= bar_length:
        if curr_pred_buy and len(trades) < max_open_trades:
            all_buys = True

            for trade in trades:
                if trade['trade_type'] != 'buy':
                    all_buys = False
                    break

            if all_buys:
              # pullback = None
              # u = i - 4

              # while u >= 0:
              #     if df.loc[df.index[u], 'fractal'] == 1:
              #         curr_pullback = float(df.loc[df.index[u], 'Bid_Low'])
              #         curr_fractal_distance = curr_ao - curr_pullback

              #         if curr_fractal_distance >= fractal_distance:
              #             pullback = curr_pullback
              #             break

              #     u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_ask_open)
                  stop_loss = round(pullback - pullback_cushion, 5)
                  # stop_loss = round(pullback - pullback_cushion, 3)

                  if stop_loss < open_price:
                      curr_pips_to_risk = open_price - stop_loss

                      if curr_pips_to_risk <= max_pips_to_risk:
                          # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                          stop_gain = round(open_price + 0.0050, 5)
                          # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                            'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                            'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_buys += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        # elif beep_boop2 == 2 and beep_boop1 == 2 and curr_mh < ema200 and enough_volatility and len(trades) < max_open_trades and curr_bh >= max(previous_bid_highs) and curr_bar_length <= bar_length:
        elif False:
            all_sells = True

            for trade in trades:
                if trade['trade_type'] != 'sell':
                    all_sells = False
                    break

            if all_sells:
              # pullback = None
              # u = i - 4

              # while u >= 0:
              #     if df.loc[df.index[u], 'fractal'] == 2:
              #         curr_pullback = float(df.loc[df.index[u], 'Ask_High'])
              #         curr_fractal_distance = curr_pullback - curr_bo

              #         if curr_fractal_distance >= fractal_distance:
              #             pullback = curr_pullback
              #             break

              #     u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_bid_open)
                  stop_loss = round(pullback + pullback_cushion, 5)
                  # stop_loss = round(pullback + pullback_cushion, 3)

                  if stop_loss > open_price:
                      curr_pips_to_risk = stop_loss - open_price

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                  'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                  'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_sells += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        if process_trade:
          iterate_thru_small_data(trades[-1])

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak

In [25]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.5, 1.6, 1.7, 1.8, 1.9, 2]
# risk_reward_ratio_vals = [2]
pullback_cushion_vals = [0, 5, 10]
# pullback_cushion_vals = [0]
# atr_percentages = [0.20, 0.25, 0.30, 1]
atr_percentages = [0.25]
# last_x_bars_vals = [2, 3]
last_x_bars_vals = [2]
# fractal_distances = [0, 5, 10]
fractal_distances = [10]
# bar_lengths = [5, 10]
bar_lengths = [5]
ema_vals = ['ema15', 'ema20', 'ema25', 'ema30', 'ema50']
# ema_vals = ['ema50']
max_open_trades_vals = [1]
n_possibilities = len(risk_reward_ratio_vals) * len(pullback_cushion_vals) * len(atr_percentages) * len(last_x_bars_vals) * len(fractal_distances) * len(bar_lengths) * len(ema_vals) * len(max_open_trades_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
  for pullback_cushion in pullback_cushion_vals:
    for atr_percentage in atr_percentages:
      for last_x_bars in last_x_bars_vals:
        for fractal_distance in fractal_distances:
          for bar_length in bar_lengths:
            for ema_val in ema_vals:
              for max_open_trades_val in max_open_trades_vals:
                all_combos.append((risk_reward_ratio, pullback_cushion, atr_percentage, last_x_bars, fractal_distance, bar_length, ema_val, max_open_trades_val))

# percentage_to_try = 0.15
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_pullback_cushion = None
best_atr_percentage = None
best_last_x_bars_val = None
best_fractal_distance = None
best_bar_length = None
best_ema_val = None
best_max_open_trades_val = None
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, pullback_cushion, atr_percentage, last_x_bars, fractal_distance, bar_length, ema_val, max_open_trades_val in combos_to_try:
  reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak = run_simulation(risk_reward_ratio, pullback_cushion, atr_percentage, last_x_bars, fractal_distance, bar_length, ema_val, max_open_trades_val)
  runs_finished += 1

  print(reward)
  print('Num buys: ' + str(n_sells))
  print('Num sells: ' + str(n_buys))
  print('Num trades: ' + str(n_buys + n_sells))
  print('Num wins: ' + str(n_wins))
  print('Num losses: ' + str(n_losses))
  print('Win streak: ' + str(win_streak))
  print('Loss streak: ' + str(loss_streak))
  print('Remaining runs: ' + str(n_runs - runs_finished))

  if reward > best_reward:
    best_reward = reward
    best_risk_reward = risk_reward_ratio
    best_pullback_cushion = pullback_cushion
    best_atr_percentage = atr_percentage
    best_last_x_bars_val = last_x_bars
    best_fractal_distance = fractal_distance
    best_bar_length = bar_length
    best_ema_val = ema_val
    best_max_open_trades_val = max_open_trades_val

  print('Best reward so far: ' + str(best_reward))
  print()

Num runs: 90

-307.9999999999977
Num buys: 0
Num sells: 1
Num trades: 1
Num wins: 0
Num losses: 1
Win streak: 0
Loss streak: 1
Remaining runs: 89
Best reward so far: -307.9999999999977

-331.99999999999494
Num buys: 0
Num sells: 1
Num trades: 1
Num wins: 0
Num losses: 1
Win streak: 0
Loss streak: 1
Remaining runs: 88
Best reward so far: -307.9999999999977

-347.9999999999933
Num buys: 0
Num sells: 1
Num trades: 1
Num wins: 0
Num losses: 1
Win streak: 0
Loss streak: 1
Remaining runs: 87
Best reward so far: -307.9999999999977

-343.0000000000004
Num buys: 0
Num sells: 1
Num trades: 1
Num wins: 0
Num losses: 1
Win streak: 0
Loss streak: 1
Remaining runs: 86
Best reward so far: -307.9999999999977

-321.9999999999961
Num buys: 0
Num sells: 1
Num trades: 1
Num wins: 0
Num losses: 1
Win streak: 0
Loss streak: 1
Remaining runs: 85
Best reward so far: -307.9999999999977

-2.500000000005276
Num buys: 0
Num sells: 1
Num trades: 1
Num wins: 0
Num losses: 1
Win streak: 0
Loss streak: 1
Remaining ru